# INSTALL: packages

In [15]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [16]:
pip install graphviz

You should consider upgrading via the 'c:\Users\info\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [17]:
import os
import pickle
#from ipynb.fs.full.new_WalletClustering_MIH_fast import iterMultiInputClustering
from graphviz import Digraph
import pandas as pd

# EXECUTE: WalletClustering_AddressIdentification notebook

In [ ]:
%run WalletClustering_AddressIdentification.ipynb # also includes running WalletClustering_neo4jConnect notebook
# methods & variables of notebook can be referenced

# EXECUTE: new-WalletClustering_MIH_fast notebook

In [ ]:
# first ensure that only methods are compiled!
# %run new-WalletClustering_MIH_fast.ipynb 
# methods & variables of notebook can be referenced

# DEFINE: Make graph of flows

## create blacklist of associations

In [19]:
def createAssocBlacklist():
    # use existing assocBlacklist if exists
    if not os.path.exists('output\\assocBlacklist.pickle'):
        assocBlacklistDf = pickle.load(open('output\\blacklistAddresses.pickle', 'rb'))
        assocBlacklistDf = assocBlacklistDf.drop_duplicates('association', keep='first')
        assocBlacklist = assocBlacklistDf['association'].tolist()

        with open('output\\assocBlacklist.pickle', 'wb') as export:
            pickle.dump(set(assocBlacklist), export)

    assocBlacklist = pickle.load(open('output\\assocBlacklist.pickle', 'rb'))

    # this line is necessary to fix unhashable type list in assocBlacklistCheck function
    # See here: http://net-informations.com/python/iq/unhashable.htm
    tuple_assocBlacklist = tuple(assocBlacklist)
    return tuple_assocBlacklist
createAssocBlacklist()

('Bitcoin.de',
 'Bleutrade.com',
 'Bit-x.com',
 'BitKonan.com',
 'BitBargain.co.uk',
 'Indacoin.com',
 'Bitstamp.net',
 'CoinChimp.com',
 'Vircurex.com',
 'bitzlato.com',
 'luno.com',
 'bitbargain.co.uk',
 'TheRockTrading.com',
 'quadrigacx.com',
 'bitcurex.com',
 'Bitfinex.com',
 'Cryptsy.com',
 'BitBay.net',
 'lakebtc.com',
 'Banx.io',
 'bitcoin.de',
 'Poloniex.com',
 'Cex.io',
 'SimpleCoin.cz',
 'BitVC.com',
 'fybsg.com',
 'coinhako.com',
 'SpectroCoin.com',
 'coingi.com',
 'bitkonan.com',
 'LakeBTC.com',
 'QuadrigaCX.com',
 'BtcMarkets.net',
 'OrderBook.net',
 'LocalBitcoins.com',
 'Bitso.com',
 'Exchanging.ir',
 'Bitcurex.com',
 'bitbay.net',
 'BitcoinVietnam.com.vn',
 '796.com',
 'CampBX.com',
 'AnxPro.com',
 'BX.in.th',
 'bitstamp.net',
 'Luno.com',
 'kraken.com',
 'HappyCoins.com',
 'ExchangeMyCoins.com',
 'Cryptonit.net',
 'mercadobitcoin.com.br',
 'virwox.com',
 'Paxful.com',
 'MaiCoin.com',
 'ChBtc.com',
 'Ccedk.com',
 'LiteBit.eu',
 'bx.in.th',
 'bleutrade.com',
 'Zyado.com

## check for blacklisted association

In [5]:
#check if association is part of blacklisted associations
def assocBlacklistCheck(association, assocBlacklist):
    if association in assocBlacklist:
        return True
    else:
        return False

## query for input address association

In [10]:
def getAssoc(address):
    # return association of input address
    assocTemplate = '''
    MATCH (a:Address {address: "%s"})
    RETURN a.association
    '''

    association = conn.query(assocTemplate % address, db='neo4j')
    
    if association is None:
        # run MI heuristic on address and query again
        addresses, association = iterMultiInputClustering_chunks(address) #should also flag new association in neo4J
        return association

    else:
        assocBlacklist = createAssocBlacklist()
        if assocBlacklistCheck(association, assocBlacklist):
            #throw exception in case association is blacklisted
            raise Exception('Blacklist')
        else:
            return association  

[<Record a.association='yYiOdGN2a2TGR8GBwrMtk1CkMAFUcaan'>]

## gather input and output addresses to association

In [6]:
def get_Inputs(query_Input):
    response = conn.query(query_Input, db='neo4j')
    rows_list = []
    for record in response:
        dict1 = {}
        new_row = {'InputAddresses':record[0]._properties["address"],
        'Value_Received_from_Inp':record[1], 'Value_Send_from_Inp':record[2], 
        'Transaction_ID':record[3], 'Association':record[4]}
        dict1.update(new_row)
        rows_list.append(dict1)
    input_df = pd.DataFrame(rows_list) 
    return input_df  

In [7]:
def get_Outputs(query_Output):
    response = conn.query(query_Output, db='neo4j')
    rows_list = []
    for record in response:
        dict1 = {}
        new_row = {'Output_Addresses':record[0]._properties["address"],
        'Value_send_to_Outp':record[1], 'Value_rec_from_Outp':record[2],
        'Transaction_ID':record[3], 'Association':record[4]}
        dict1.update(new_row)
        rows_list.append(dict1)
    output_df = pd.DataFrame(rows_list) 
    return output_df

In [11]:
#get all addresses and their associations that are either output or input to an association (user)
def getInputs_Outputs(association):
    
    query_for_Outputs = """
    Match (a:Address {association: "%s"})-[send:SENDS]->(t:Transaction)-[r:RECEIVES]->(out:Address)
    return out as output_Addresses, send.value as Value_send_to_Outp,r.value as Value_rec_from_Outp,
     t.txid as Transaction_ID, out.association as Association"""

    userOutput = get_Outputs(query_for_Outputs % association)

    query_for_Inputs = """
    Match (input:Address)-[s:SENDS]->(t:Transaction)-[r:RECEIVES]->(a:Address {association: "%s"})
    return input as InputAddresses, r.value as Value_Received_from_Inp,s.value as Value_Send_from_Inp,
    t.txid as Transaction_ID, input.association as Association"""

    
    userInput = get_Inputs(query_for_Inputs % association)

    return userOutput, userInput

(                     Output_Addresses  Value_send_to_Outp  \
 0  1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA              500000   
 1  12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8              500000   
 2  1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA              494281   
 3  12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8              494281   
 4  1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA              489847   
 5  12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8              489847   
 6  1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA              485413   
 7  12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8              485413   
 8  1FoWyxwPXuj4C6abqwhjDWdz6D4PZgYRjA              478028   
 9  12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8              478028   
 
    Value_rec_from_Outp                                     Transaction_ID  \
 0                  546  3a785a153bfd7d77bdf7b8dcce6ba24feeb9b1bffdb723...   
 1               494281  3a785a153bfd7d77bdf7b8dcce6ba24feeb9b1bffdb723...   
 2                  546  1b1be7bab9f905ffcf083e3483705f0e621a9518930b69...   
 3  

## query for input and output address asociations

In [14]:
def buildFlowGraph(inputAddress):
    try:
        association = getAssoc(inputAddress)
    except:
        raise Exception('FU') # adjust to use case

    outputs, inputs = getInputs_Outputs(association)
   
    outputAssocs = []
    for o in outputs.rows():
        if o['Association'] not None:
            outputAssocs.append(o['Association'])
        else:
            outputAssocs.append(getAssoc(o['OutputAddresses']))
    outputAssocs = set(outputAssocs)
    
    inputAssocs = []
    for i in inputs.rows():
        if i['Association'] not None:
            inputAssocs.append(i['Association'])
        else:
            inputAssocs.append(getAssoc(i['InputAddresses']))
    inputAssocs = set(inputAssocs)

    # build graph (adjust to use case)
    dot = Digraph()
    dot.node(association, association)
        
    for outNode in outputAssocs:
        dot.node(outNode, outNode)
        dot.edge(association, outNode)

    for inNode in inputAssocs:
        dot.node(inNode, inNode)
        dot.edge(inNode, association)
        
    return dot

## include transaction information in graph

In [15]:
def getUserTransactionData(association, outputNodes, inputNodes):
    userOutputTransactionDataTemplate = '''
        MATCH (u:Address{association:"%s"})-[:SENDS]->(t:Transaction)-[:SENDS]->(o:Address{association:"%s"}),
        (u:Address)-[:SENDS]->(t:Transaction)-[:SENDS]->(o:Address)
        RETURN DISTINCT o
        ''' 

# RUN: Make graph of flows

In [16]:
inputAddress = ''
getAssoc("12sDU3FyYJXc2oRzE6XXuuhVHCBJvaoCC8")
getInputs_Outputs("yYiOdGN2a2TGR8GBwrMtk1CkMAFUcaan")

In [17]:
graph = buildFlowGraph(inputAddress)
graph.render(view=True)

Exception: FU